In [4]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img

# Define the data generator
data_gen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load all images in the directory
data_dir = 'dataset/train/cercospera'
files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]
x_train = np.array([np.array(load_img(f)) for f in files])

# Generate augmented data
data_gen.fit(x_train)
flow = data_gen.flow(x_train, batch_size=32)

# Save the augmented images
new_images_dir = 'new_images'
if not os.path.exists(new_images_dir):
    os.makedirs(new_images_dir)

for i, batch in enumerate(flow):
    for j in range(batch.shape[0]):
        img = array_to_img(batch[j], scale=True)
        img.save(os.path.join(new_images_dir, 'augmented_image_{}_{}.jpg'.format(i, j)))
        if i * 32 + j >= x_train.shape[0]:
            break
    if i * 32 >= x_train.shape[0]:
        break

ImportError: cannot import name 'array_to_img' from 'keras.preprocessing.image' (C:\Users\rafae\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\preprocessing\image.py)

In [7]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import save_img, img_to_array

# Define the data generator
data_gen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load all images in the directory
data_dir = 'dataset/train/cercospera'
files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]
x_train = np.array([img_to_array(load_img(f)) for f in files])

# Generate augmented data
data_gen.fit(x_train)
flow = data_gen.flow(x_train, batch_size=32)

# Save the augmented images
new_images_dir = 'new_images'
if not os.path.exists(new_images_dir):
    os.makedirs(new_images_dir)

for i, batch in enumerate(flow):
    for j in range(batch.shape[0]):
        img = batch[j].astype('uint8')
        save_img(os.path.join(new_images_dir, 'augmented_image_{}_{}.jpg'.format(i, j)), img)
        if i * 32 + j >= x_train.shape[0]:
            break
    # if i * 32 >= x_train.shape[0]:
        break


ImportError: cannot import name 'save_img' from 'keras.preprocessing.image' (C:\Users\rafae\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\preprocessing\image.py)

In [6]:
# from random import seed
from tensorflow import keras
import tensorflow as tf

ds = keras.preprocessing.image_dataset_from_directory(
    "dataset/train",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    batch_size=1,
    image_size=(512, 512),
    shuffle=False,
)

class_names = ds.class_names

def augment(image, label):
    seed = tf.random
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.5)
    image = tf.image.random_contrast(image, lower=0.5, upper=1)
    image = tf.image.random_saturation(image, lower=0.5, upper=1)
    image = tf.image.random_hue(image, max_delta=0.1)
    image = tf.image.random_crop(image, size=[1, 384, 384, 3])
    image = tf.image.central_crop(image, central_fraction=0.75)
    image = tf.image.rot90(image, k=1)
    return image, label

ds = ds.map(augment)
# 256 + 128 = 384

def save_image(image, path_dest, name):
    image = image.numpy().astype("uint8")
    image = np.squeeze(image, axis=0)
    image = tf.image.encode_jpeg(image, quality=100, optimize_size=True, chroma_downsampling=True)
    tf.io.write_file(path_dest + "/" + name, image)

i = 0
# for i in range(ds.cardinality().numpy()):
for image, label in ds:
    i += 1
    # image, label = next(iter(ds))
    class_name = class_names[label.numpy().argmax()]
    # print(class_name)
    filename = f"{i}.jpg"
    save_image(image, class_name, filename)

Found 110 files belonging to 2 classes.


NameError: name 'np' is not defined

In [70]:
image, label = next(iter(ds))
label

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1., 0.]], dtype=float32)>

In [24]:
from tensorflow import keras
import tensorflow as tf
import numpy as np

def augment(image, label):
    image = tf.image.rot90(image, k=1) if tf.random.uniform(()) > 0.5 else image
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=1.0)
    image = tf.image.random_saturation(image, lower=0.75, upper=1.25)
    image = tf.image.random_crop(image, size=[1, 448, 448, 3])
    return image, label


def save_image(image, path_dest, name):
    image = image.numpy().astype("uint8")
    image = np.squeeze(image, axis=0)
    image = tf.image.encode_jpeg(image, quality=100, optimize_size=False, chroma_downsampling=False)
    tf.io.write_file(path_dest + "/" + name, image)


for i in range(7):
    ds = keras.preprocessing.image_dataset_from_directory(
            "images",
            labels="inferred",
            label_mode="categorical",
            class_names=None,
            image_size=(512, 512),
            shuffle=False,
            batch_size=1,
        )

    class_names = ds.class_names

    ds = ds.map(augment)

    for j, (image, label) in enumerate(ds):
        file_index  = (i * 64) + j
        class_name = class_names[label.numpy().argmax()]
        filename = f"{file_index}.jpg"
        save_image(image, class_name, filename)

Found 64 files belonging to 1 classes.
Found 64 files belonging to 1 classes.
Found 64 files belonging to 1 classes.
Found 64 files belonging to 1 classes.
Found 64 files belonging to 1 classes.
Found 64 files belonging to 1 classes.
Found 64 files belonging to 1 classes.


In [8]:
import tensorflow as tf

seed = np.random.randint(0, 1000)

<module 'tensorflow._api.v2.random' from 'C:\\Users\\rafae\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\tensorflow\\_api\\v2\\random\\__init__.py'>

In [ ]:
import os
import numpy as np
import cv2
import imgaug as ia
from imgaug import augmenters as iaa

# Caminho para o diretório com as imagens
path = "dataset/train/cercospera"

# Carregar as imagens
images = [cv2.imread(os.path.join(path, f)) for f in os.listdir(path)]

# Criar uma sequência de aumento de dados
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # flip horizontal com 50% de probabilidade
    iaa.Affine(rotate=45), # rotacionar em 45 graus
    iaa.Multiply((1.2, 1.5)) # aumentar o brilho com uma multiplicação aleatória entre 1.2 e 1.5
])

# Armazenar as imagens aumentadas em uma lista
images_aug = []
for i, image in enumerate(images):
    image_aug = seq.augment_image(image)
    images_aug.append(image_aug)
    cv2.imwrite("image_aug_{0}.jpg".format(i), image_aug)
    print("Image{0}:".format(i))



In [ ]:
import os
import tensorflow as tf
import cv2
import numpy as np

# Caminho para o diretório com as imagens
path = "dataset/train/cercospera"


# Criar o diretório para salvar as imagens aumentadas, se ele ainda não existir
if not os.path.exists("augmented_images"):
    os.makedirs("augmented_images")

# Criar um gerador de dados
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Carregar as imagens
# images = [cv2.imread(os.path.join(path, f)) for f in os.listdir(path)]
# Carregar as imagens
images = [cv2.imread(os.path.join(path, f), cv2.IMREAD_COLOR) for f in os.listdir(path)]

# Verificar se as imagens estão em formato RGB
for image in images:
    if len(image.shape) != 3 or image.shape[2] != 3:
        raise Exception("As imagens devem estar em formato RGB")

# Redimensionar as imagens para as dimensões especificadas
img_height, img_width = 256, 256
images = [cv2.resize(img, (img_height, img_width)) for img in images]

# Normalizar as imagens para o intervalo [0,1]
images = np.array(images, dtype="float32") / 255.0

# Gerar imagens aumentadas com base nas imagens originais
i = 0
for batch in datagen.flow(images, batch_size=1, save_to_dir='augmented_images', save_prefix='image', save_format='jpg'):
    i += 1
    if i >= len(images):
        break


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data generator
data_gen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=keras.applications.resnet50.preprocess_input
)

# Load all images in the directory
data_dir = 'dataset/train/cercospera'
files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]
x_train = np.array([np.expand_dims(keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(f, target_size=(224,224))), axis=0) for f in files])

# Generate augmented data
data_gen.fit(x_train)
flow = data_gen.flow(x_train, batch_size=32, save_to_dir='new_images', save_prefix='augmented_image', save_format='jpg', seed=1)

# Save the augmented images
new_images_dir = 'new_images'
if not os.path.exists(new_images_dir):
    os.makedirs(new_images_dir)

for i, batch in enumerate(flow):
    for j in range(batch.shape[0]):
        img = batch[j].astype('uint8')
        if i * 32 + j >= x_train.shape[0]:
            break


In [ ]:
import os
import shutil
import random

# Diretório com as pastas de classes
images_dir = ["dataset/test/cercospera", "dataset/test/soybean_rust"]

# Porcentagem de imagens para cada conjunto
train_percentage = 0.6
val_percentage = 0.2

for image_dir in images_dir:
  # Cria os diretórios para os conjuntos de dados
  train_dir = os.path.join(image_dir, 'train32')
  os.makedirs(train_dir, exist_ok=True)

  val_dir = os.path.join(image_dir, 'val32')
  os.makedirs(val_dir, exist_ok=True)

  test_dir = os.path.join(image_dir, 'test32')
  os.makedirs(test_dir, exist_ok=True)

  # Obtém a lista de todas as imagens no diretório de imagens
  image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

  # Embaralha a lista de imagens
  random.shuffle(image_files)

  # Calcula o número de imagens para cada conjunto
  train_count = int(train_percentage * len(image_files))
  val_count = int(val_percentage * len(image_files))

  # Copia as imagens para os diretórios correspondentes
  for i, image_file in enumerate(image_files[:train_count]):
      shutil.copy2(os.path.join(image_dir, image_file), os.path.join(train_dir, image_file))

  for i, image_file in enumerate(image_files[train_count:train_count + val_count]):
      shutil.copy2(os.path.join(image_dir, image_file), os.path.join(val_dir, image_file))

  for i, image_file in enumerate(image_files[train_count + val_count:]):
      shutil.copy2(os.path.join(image_dir, image_file), os.path.join(test_dir, image_file))
